### Imports

In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from datetime import datetime
from bokeh.palettes import Spectral3
from tabulate import tabulate
from bokeh.transform import factor_cmap
import js2py
import math
from concurrent.futures import ProcessPoolExecutor as PoolExecutor
import requests

# convenience
output_notebook()
pd.set_option('display.max_colwidth', -1)

Loading BokehJS ...

### Globals

In [3]:
API = 'https://explorer.helium.foundation/api'
NUM_CHALLENGES = 100
NUM_BLOCKS = 100
TOP = 10
animalhash = js2py.require('angry-purple-tiger')
MAX_WORKERS = 8

### Functions

In [4]:
def latest_height(api):
    '''
    Get latest block height
    '''
    r = requests.get(f'{api}/blocks?limit=1')
    if r.status_code == 200:
        data = r.json().get('data')
        return data[0]['height']
    raise Exception('Unable to get latest height')

def get_blocks(api, num_blocks=100):
    '''
    Get past `NUM_BLOCKS` number of num_blocks
    '''
    r = requests.get(f'{api}/blocks?limit={num_blocks}')
    if r.status_code == 200:
        data = r.json().get('data')
        assert len(data) == num_blocks
        return data
    raise Exception('Unable to get blocks')

def get_txn(api, block_height):
    '''
    Get transactions for given block_height
    Called from `get_transactions`
    '''
    r = requests.get(f'{api}/blocks/{block_height}/transactions')
    if r.status_code == 200:
        data = r.json().get('data')
        return data
    raise Exception('Unable to get txns')

def get_challenges(api, num_challenges=100):
    '''
    Get past `LIMIT` number of challenges
    '''
    r = requests.get(f'{api}/challenges?limit={num_challenges}')
    if r.status_code == 200:
        data = r.json().get('data')
        assert len(data) == num_challenges
        return accumulate_results(data)
    raise Exception('Unable to get challenges')

def get_transactions(start, num_blocks=100):
    '''
    Get transactions for each block
    Starting from block height `LATEST_HEIGHT` by default and ending at start - num_blocks
    '''
    heights = range(start, start-num_blocks, -1)
    with PoolExecutor(max_workers=MAX_WORKERS) as executor:
        txns = {}
        for height, i in zip(heights, executor.map(get_txn, heights)):
            txns[height] = len(i)
    return txns

def get_hotspots(api):
    '''
    Get hotspots from the api
    '''
    r = requests.get(f'{api}/hotspots')
    if r.status_code == 200:
        data = r.json().get('data')
        return data
    raise Exception('Unable to get hotspots')

def get_accounts(api):
    '''
    Get accounts from the api
    '''
    r = requests.get(f'{api}/accounts')
    if r.status_code == 200:
        data = r.json().get('data')
        return data
    raise Exception('Unable to get accounts')

def accumulate_results(challenges):
    '''
    Return a dict list of challenge_id, num_successes, num_failures and num_untested
    '''
    results = []
    for challenge in challenges:
        path = challenge['pathElements']

        result = [element['result'] for element in path]
        length = len(result)
        num_untested = result.count('untested')

        gray = True if num_untested == length else False

        if gray:
            num_successes = 0
            num_failures = 0
            num_untested = 0
            dud = 1
            success = 0
        else:
            num_successes = result.count('success')
            num_failures = result.count('failure')
            num_untested = num_untested
            dud = 0
            success = 1 if num_successes == length else 0
        results.append({'id': challenge['id'],
                        'num_successes': num_successes,
                        'num_failures': num_failures,
                        'num_untested': num_untested,
                        'dud': dud,
                        'success': success})
    return results


### Block Stats

In [5]:
blocks = get_blocks(API, NUM_BLOCKS)

In [6]:
df = pd.DataFrame(blocks)

In [7]:
print(f'Average txns past {NUM_BLOCKS} blocks: {df["txns"].mean()}')

Average txns past 100 blocks: 4.92


In [8]:
print(f'Average block time past {NUM_BLOCKS} blocks: {df["time"].mean()}')

Average block time past 100 blocks: 1565637176.02


In [9]:
max_txns = tabulate(df.max().reset_index(),tablefmt='psql', showindex='never')
print(f"Block with max txns for past {NUM_BLOCKS} blocks: \n{max_txns}")

Block with max txns for past 100 blocks: 
+--------+----------------------------------------------------+
| hash   | 1zSEn1GybDMXrF9QYDPgar4jjgsGh2aYX2xP5mQm1mfhvfUgTj |
| height | 14383                                              |
| round  | 14383                                              |
| time   | 1565641854                                         |
| txns   | 14                                                 |
+--------+----------------------------------------------------+


In [10]:
source = ColumnDataSource(df)
p = figure()

p.vbar(x='height', top='txns', width=0.8, source=source)
hover = HoverTool(tooltips=[('height', '@height'), ('txns', '@txns')])
p.add_tools(hover)
p.xaxis.axis_label = "Txns/Block"
p.yaxis.axis_label = "NumTxns"

show(p)

### Hotspot Stats

In [11]:
hotspots = get_hotspots(API)

In [12]:
df = pd.DataFrame(hotspots)
df['name'] = df.apply(lambda row: '-'.join(animalhash(row.address).lower().split()), axis=1)

In [13]:
print(f'Top {TOP} scoring hotspots:')
df.sort_values('score', ascending=False)[['name', 'score']].head(TOP)

Top 10 scoring hotspots:


,name,score
111,magic-carob-quail,0.9898
8,beautiful-onyx-platypus,0.9390
128,mean-alabaster-piranha,0.9384
98,delightful-stone-beetle,0.9353
125,mini-currant-lizard,0.9124
127,mythical-tin-hawk,0.9058
55,shambolic-rusty-cyborg,0.9010
121,joyous-burgundy-chicken,0.8936
39,bubbly-mandarin-panther,0.8765
67,teeny-cider-troll,0.8307


In [14]:
source = ColumnDataSource(df)
# p = figure()
p = figure(x_range=df['name'], y_range=[0,1], title="Hotspot Scores")

p.vbar(x='name', top='score', width=0.8, source=source)
hover = HoverTool(tooltips=[('name', '@name'), ('score', '@score')])
p.width = 900
p.add_tools(hover)
p.xaxis.axis_label = "Name"
p.yaxis.axis_label = "Score"
p.xaxis.major_label_text_font_size = "4pt"
p.xaxis.major_label_orientation = math.pi/4

show(p)

### Account stats

In [15]:
accounts = get_accounts(API)

In [16]:
df = pd.DataFrame(accounts)

In [17]:
print(f'Top {TOP} account balances:')
df.sort_values('balance', ascending=False)[['address', 'balance']].head(TOP)

Top 10 account balances:


,address,balance
36,13jL4pTLq4JQBwDoH1ZBX3VPchmeTZxQSGi4cnyXXuzPPoKxcd5,4405066432608
67,1398hLeHESZHE5jVtaLAV5fdg2vrUeZEs2B92t7TzeQTtugr8dL,3063111932691
95,14a5sytsEVKBVhGsVz2rJDkxf1cs1L89zvtFjmPtmPzrQHXPVjF,2517340037647
80,14GWyFj9FjLHzoN3aX7Tq7PL6fEg4dfWPY8CrK8b9S5ZrcKDz6S,1449883323242
83,12yyntxFfPhqXDKuEvmAWABNTRnXtYmcgS33WN4tk5YtyaBLDXo,1448084252884
47,13o9KUMcmEdhtjXRaJ5AANHLZWBoGcTmF4MFw3MjQbubjR57jXW,1320953771510
79,12z4nUiayZnbW46azcegBBB9yGkchFb1Zm7EPuh6eV1r2HT1NgC,883905831669
73,14i6nM1RbWbjQFg5NFbZS3aZJgpUGZUx5toarz6jZLPpNAb95Tq,831384743562
87,14UgUhAPq53iR8vfde6sU4vLPWRe6kZ3ioV7xaVb8hr3oafaok6,828504671852
76,135bB6B7gGrsdPNcFwvcn12KPjTktSNZvmVbvSBKubCtEgxkpPJ,804587266385


### Challenge Stats

In [18]:
challenges = get_challenges(API, NUM_CHALLENGES)

In [19]:
df = pd.DataFrame(challenges)

In [20]:
print(f"Total Successful challenges past {NUM_CHALLENGES} challenges: {sum(df['success'] == 1)}")

Total Successful challenges past 100 challenges: 19


In [21]:
print(f"Entirely dud challenges since past {NUM_CHALLENGES} challenges: {sum(df['dud'] == 1)}")

Entirely dud challenges since past 100 challenges: 23
